In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

# 1. 自作ソースコード(src)が入ったDatasetを追加
sys.path.append('/kaggle/input/my-pasture-lib')

# 2. Notebook ① の出力（Features）が入ったパス
# ※ ここは自分の Notebook ① の名前に書き換えてください
FEAT_PATH = Path("/kaggle/input/notebook-1-slug/features")

from src.config import CFG
from src.features.supervised import SupervisedEmbeddingEngine
from src.models.tabular import run_tabular_cv, MultiTargetRegressor
from src.utils.metrics import competition_metric, post_process_biomass
from src.utils.logger import seeding

seeding(CFG.SEED)

### データのロードと結合

In [ ]:
# Notebook ① からの出力を読み込む
X_emb = np.load(FEAT_PATH / "train_siglip_embeddings.npy")
X_semantic = np.load(FEAT_PATH / "train_semantic_scores.npy")
train_df = pd.read_csv(FEAT_PATH / "train_meta.csv")

# 5-Foldの分割を定義
from sklearn.model_selection import KFold
kf = KFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
train_df['fold'] = -1
for i, (_, val_idx) in enumerate(kf.split(train_df)):
    train_df.loc[val_idx, 'fold'] = i

print(f"Features loaded: Embedding {X_emb.shape}, Semantic {X_semantic.shape}")

### 学習の実行|Cross Validation

In [ ]:
from catboost import CatBoostRegressor

# エンジンとモデルの定義
feat_engine = SupervisedEmbeddingEngine(n_pca=0.80, n_pls=8, n_gmm=6)
base_model = CatBoostRegressor(
    iterations=1000, 
    learning_rate=0.05, 
    depth=6, 
    verbose=0,
    random_seed=CFG.SEED
)

# CVの実行
print("Starting CV training...")
oof_preds = run_tabular_cv(
    train_df=train_df,
    X_raw=X_emb,
    feature_engine=feat_engine,
    model_instance=base_model,
    X_semantic=X_semantic
)

# スコア確認
y_true = train_df[CFG.TARGET_NAMES].values
raw_score = competition_metric(y_true, oof_preds)
processed_oof = post_process_biomass(pd.DataFrame(oof_preds, columns=CFG.TARGET_NAMES))
final_score = competition_metric(y_true, processed_oof.values)

print(f"CV Raw Score: {raw_score:.6f}")
print(f"CV Processed Score: {final_score:.6f}")

### モデルの保存（全データでの再学習またはFoldごとの保存）
Notebook ③ で使うために、モデルをファイルとして書き出します。

In [ ]:
# ここでは例として「最後のFoldのエンジンとモデル」を保存しますが、
# 本来は各Foldのモデルを全て保存するのがベストです。
os.makedirs("/kaggle/working/models", exist_ok=True)

# 最後のFoldで学習したインスタンスを保存する例
with open("/kaggle/working/models/final_model.pkl", "wb") as f:
    pickle.dump({
        "engine": feat_engine, # fit済みの特徴量抽出器
        "model": base_model    # 学習済みモデル
    }, f)

print("Models saved to /kaggle/working/models/")